In [34]:
from dotenv import load_dotenv
import os
import json
import asyncio
from agents import Agent, function_tool, Runner, trace, input_guardrail, output_guardrail, GuardrailFunctionOutput, RunContextWrapper
from datetime import datetime, timedelta
from openai import OpenAI
import requests
from typing import Dict, Union, List
import re


load_dotenv(override=True)

# Initialize OpenAI client - this was missing!
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [35]:
@function_tool
def parse_flight_query(user_query: str) -> Dict[str, any]:
    """
    SDK-reference flight query parser using exact Amadeus templates for endpoint-specific structures.
    
    Args:
        user_query: Natural language flight query
        
    Returns:
        Dictionary with structured Amadeus API command using SDK templates
    """
    
    print(f"\n🔍 Analyzing flight query: '{user_query}'")
    
    tomorrow = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
    today = datetime.now().strftime("%Y-%m-%d")
    
    # SDK-based templates with exact structures
    parser_instructions = f"""You are an Amadeus SDK expert. Generate EXACT request structures based on official SDK templates.

CONTEXT:
- Today: {today}
- Tomorrow: {tomorrow}

AMADEUS SDK ENDPOINT TEMPLATES:

1. FLIGHT SEARCH - amadeus.shopping.flightOffersSearch.get(params)
   Endpoint: "shopping/flight-offers"
   Method: "GET"
   Template: {{
     "originLocationCode": "ARN",
     "destinationLocationCode": "LHR",
     "departureDate": "2025-10-10",
     "adults": "1",
     "travelClass": "ECONOMY",
     "max": "10"
   }}

2. FLIGHT AVAILABILITIES - amadeus.shopping.availability.flightAvailabilities.post(body)
   Endpoint: "shopping/availability/flight-availabilities"  
   Method: "POST"
   Template: {{
     "originDestinations": [{{
       "id": "1",
       "originLocationCode": "ARN",
       "destinationLocationCode": "LHR",
       "departureDateTime": {{
         "date": "2025-10-10"
       }}
     }}],
     "travelers": [{{
       "id": "1",
       "travelerType": "ADULT"
     }}],
     "sources": ["GDS"]
   }}

3. FLIGHT PRICING - amadeus.shopping.flightOffers.pricing.post(body)
   Endpoint: "shopping/flight-offers/pricing"
   Method: "POST"  
   Template: {{
     "data": {{
       "type": "flight-offers-pricing",
       "flightOffers": [{{
         "type": "flight-offer",
         "id": "1"
       }}]
     }}
   }}

INTENT MATCHING:
- "search flights", "find flights", "flight options" → Template 1 (GET)
- "booking classes", "seat availability", "available seats" → Template 2 (POST)
- "confirm price", "pricing", "check price" → Template 3 (POST)

PARAMETER MAPPING:
- Stockholm/ARN: ARN
- London: LHR  
- Hanoi: HAN
- SAS: SK
- Air China: CA
- Turkish: TK

RESPONSE FORMAT - Use EXACT SDK template structure:
{{
  "user_intent": "search_flights|booking_classes|price_confirmation",
  "query_type": "flight_search|availability_check|price_confirmation",
  "amadeus_command": {{
    "endpoint": "exact-endpoint-path",
    "method": "GET|POST",
    "parameters": {{
      // COPY EXACT TEMPLATE STRUCTURE
      // For GET: flat parameters
      // For POST: nested SDK structure
    }}
  }},
  "reasoning": "Selected endpoint based on SDK method signature",
  "filled_defaults": [],
  "user_provided": []
}}

CRITICAL: 
- COPY the exact SDK template structure
- For booking classes queries, use Template 2 EXACTLY
- Do NOT modify the nested structure
- Return ONLY valid JSON

EXAMPLE for "booking classes ARN to LHR":
{{
  "user_intent": "booking_classes",
  "query_type": "availability_check", 
  "amadeus_command": {{
    "endpoint": "shopping/availability/flight-availabilities",
    "method": "POST",
    "parameters": {{
      "originDestinations": [{{
        "id": "1",
        "originLocationCode": "ARN",
        "destinationLocationCode": "LHR", 
        "departureDateTime": {{
          "date": "{tomorrow}"
        }}
      }}],
      "travelers": [{{
        "id": "1",
        "travelerType": "ADULT"
      }}],
      "sources": ["GDS"]
    }}
  }}
}}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": parser_instructions},
                {"role": "user", "content": user_query}
            ]
        )
        
        result_text = response.choices[0].message.content.strip()
        if result_text.startswith("```json"):
            result_text = result_text.replace("```json", "").replace("```", "").strip()
        if result_text.startswith("```"):
            result_text = result_text.replace("```", "").strip()
        
        parsed_result = json.loads(result_text)
        
        print(f"🎯 Intent: {parsed_result.get('user_intent')}")
        print(f"🔗 Endpoint: {parsed_result['amadeus_command']['endpoint']}")
        print(f"⚡ Method: {parsed_result['amadeus_command']['method']}")
        
        # Validate SDK structure for flight-availabilities
        if 'flight-availabilities' in parsed_result['amadeus_command']['endpoint']:
            params = parsed_result['amadeus_command']['parameters']
            sdk_keys = ['originDestinations', 'travelers', 'sources']
            for key in sdk_keys:
                if key in params:
                    print(f"✅ SDK structure: {key} present")
                else:
                    print(f"❌ Missing SDK key: {key}")
        
        return parsed_result
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON error: {e}")
        raise ValueError(f"Failed to parse JSON: {result_text}")
    except Exception as e:
        print(f"❌ Parse error: {e}")
        raise

In [36]:
# Updated API Configuration for flexible endpoint handling
AMADEUS_BASE_URL_V1 = "https://test.api.amadeus.com/v1"
AMADEUS_BASE_URL_V2 = "https://test.api.amadeus.com/v2"
AMADEUS_AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"

In [37]:
class AmadeusAPIClient:
    def __init__(self, client_id: str, client_secret: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.access_token = None
        self.token_expires_at = None
    
    def get_access_token(self) -> str:
        """Get or refresh Amadeus API access token"""
        if self.access_token and self.token_expires_at and datetime.now() < self.token_expires_at:
            return self.access_token
        
        print("🔑 Getting new Amadeus access token...")
        
        auth_data = {
            'grant_type': 'client_credentials',
            'client_id': self.client_id,
            'client_secret': self.client_secret
        }
        
        response = requests.post(AMADEUS_AUTH_URL, data=auth_data)
        
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
            # Set expiration time (subtract 60 seconds for safety)
            expires_in = token_data.get('expires_in', 3600)
            self.token_expires_at = datetime.now() + timedelta(seconds=expires_in - 60)
            print("✅ Access token obtained successfully")
            return self.access_token
        else:
            raise Exception(f"Failed to get Amadeus access token: {response.status_code} - {response.text}")


In [38]:
# Initialize Amadeus client
amadeus_client = AmadeusAPIClient(
    client_id=os.environ.get('AMADEUS_API_KEY'),  # Changed from AMADEUS_CLIENT_ID
    client_secret=os.environ.get('AMADEUS_API_SECRECT')  # Changed from AMADEUS_CLIENT_SECRET, note the typo in .env
)

In [39]:
@function_tool(strict_mode=False)
def execute_amadeus_query(parsed_query_json: str) -> str:
    """
    Intelligent Amadeus API executor with retry logic for different base URLs and date adjustments.
    
    Args:
        parsed_query_json: JSON string containing structured Amadeus API command from query parser
        
    Returns:
        JSON string containing the full response from Amadeus API
    """
    
    print(f"\n🚀 Executing Amadeus API query...")
    
    try:
        # Parse JSON input
        parsed_query = json.loads(parsed_query_json)
        print(f"🎯 User intent: {parsed_query.get('user_intent', 'unknown')}")
        print(f"📊 Query type: {parsed_query.get('query_type', 'unknown')}")
        
        # Extract command details
        amadeus_command = parsed_query.get('amadeus_command', {})
        endpoint = amadeus_command.get('endpoint', '')
        method = amadeus_command.get('method', 'GET').upper()
        parameters = amadeus_command.get('parameters', {}).copy()  # Make a copy for modifications
        
        print(f"🔗 Endpoint: {endpoint}")
        print(f"⚡ Method: {method}")
        print(f"📋 Parameters: {parameters}")
        
        # Validate required data
        if not endpoint:
            return json.dumps({
                "status": "error",
                "error": "Missing endpoint in query"
            })
        
        # Check credentials - if missing, provide intelligent mock responses
        if not amadeus_client.client_id or not amadeus_client.client_secret:
            mock_response = generate_mock_response(parsed_query, endpoint, parameters)
            return json.dumps(mock_response)
        
        # Try to get access token
        try:
            access_token = amadeus_client.get_access_token()
        except Exception as auth_error:
            print(f"🔑 Authentication failed: {auth_error}")
            mock_response = generate_mock_response(parsed_query, endpoint, parameters, auth_error=str(auth_error))
            return json.dumps(mock_response)
        
        # Retry logic with different strategies
        max_retries = 3
        retry_strategies = [
            {"base_url": AMADEUS_BASE_URL_V2, "endpoint_clean": endpoint.replace('v1/', '').replace('v2/', ''), "description": "v2 API"},
            {"base_url": AMADEUS_BASE_URL_V1, "endpoint_clean": endpoint.replace('v1/', '').replace('v2/', ''), "description": "v1 API"},
            {"base_url": AMADEUS_BASE_URL_V2, "endpoint_clean": endpoint.replace('v1/', '').replace('v2/', ''), "description": "v2 API with adjusted date", "adjust_date": True}
        ]
        
        headers = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}
        
        for attempt in range(max_retries):
            strategy = retry_strategies[attempt]
            current_params = parameters.copy()
            
            # Adjust date if strategy requires it
            if strategy.get('adjust_date') and 'departureDate' in current_params:
                from datetime import datetime, timedelta
                try:
                    current_date = datetime.strptime(current_params['departureDate'], '%Y-%m-%d')
                    adjusted_date = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
                    current_params['departureDate'] = adjusted_date
                    print(f"🔄 Adjusting date to: {adjusted_date}")
                except:
                    pass  # Keep original date if parsing fails
            
            full_url = f"{strategy['base_url']}/{strategy['endpoint_clean']}"
            
            print(f"🔄 Attempt {attempt + 1}/3: Trying {strategy['description']}")
            print(f"📡 Making {method} request to: {full_url}")
            
            try:
                if method == 'GET':
                    response = requests.get(full_url, headers=headers, params=current_params)
                else:
                    response = requests.post(full_url, headers=headers, json=current_params)
                
                print(f"📊 Response status: {response.status_code}")
                
                # Success case
                if response.status_code == 200:
                    result = {
                        "status": "success",
                        "amadeus_response": response.json(),
                        "query_info": {
                            "endpoint": strategy['endpoint_clean'],
                            "method": method,
                            "parameters": current_params,
                            "user_intent": parsed_query.get('user_intent', 'unknown'),
                            "successful_attempt": attempt + 1,
                            "strategy_used": strategy['description']
                        }
                    }
                    print(f"✅ API call successful on attempt {attempt + 1}")
                    return json.dumps(result)
                
                # Error cases - continue to next attempt if not last
                else:
                    print(f"❌ Attempt {attempt + 1} failed: {response.status_code}")
                    if attempt < max_retries - 1:
                        print(f"🔄 Retrying with different strategy...")
                        continue
                    else:
                        # Final attempt failed
                        error_result = {
                            "status": "amadeus_api_error",
                            "message": f"Amadeus API returned an error after {max_retries} attempts",
                            "final_status_code": response.status_code,
                            "final_error": response.text,
                            "attempts_made": max_retries,
                            "query_info": {
                                "endpoint": strategy['endpoint_clean'],
                                "method": method,
                                "parameters": current_params
                            }
                        }
                        print(f"❌ All {max_retries} attempts failed. Amadeus API returned an error.")
                        return json.dumps(error_result)
                        
            except requests.RequestException as req_error:
                print(f"❌ Request error on attempt {attempt + 1}: {req_error}")
                if attempt < max_retries - 1:
                    continue
                else:
                    error_result = {
                        "status": "request_error",
                        "message": f"Request failed after {max_retries} attempts",
                        "error": str(req_error)
                    }
                    return json.dumps(error_result)
            
    except Exception as e:
        print(f"💥 Execution error: {str(e)}")
        error_result = {
            "status": "execution_error",
            "error": str(e)
        }
        return json.dumps(error_result)

def generate_mock_response(parsed_query: dict, endpoint: str, parameters: dict, auth_error: str = None) -> dict:
    """Generate intelligent mock responses based on the endpoint type"""
    
    user_intent = parsed_query.get('user_intent', 'unknown')
    query_type = parsed_query.get('query_type', 'unknown')
    
    base_response = {
        "status": "mock_success" if not auth_error else "auth_error",
        "message": "Using mock data - credentials not available" if not auth_error else f"Authentication failed: {auth_error}",
        "query_info": {
            "endpoint": endpoint,
            "method": parsed_query.get('amadeus_command', {}).get('method', 'GET'),
            "parameters": parameters,
            "user_intent": user_intent
        }
    }
    
    # Generate endpoint-specific mock data
    if 'flight-offers' in endpoint and 'pricing' not in endpoint:
        # Flight search mock data
        base_response["mock_flight_data"] = {
            "data": [{
                "type": "flight-offer",
                "id": "1",
                "itineraries": [{
                    "segments": [{
                        "departure": {
                            "iataCode": parameters.get("originLocationCode", "ARN"),
                            "at": f"{parameters.get('departureDate', '2025-10-10')}T10:00:00"
                        },
                        "arrival": {
                            "iataCode": parameters.get("destinationLocationCode", "HAN"),
                            "at": f"{parameters.get('departureDate', '2025-10-10')}T18:30:00"
                        },
                        "carrierCode": "SK",
                        "number": "123"
                    }]
                }],
                "price": {"currency": "EUR", "total": "299.00"}
            }],
            "meta": {"count": 1}
        }
    elif 'flight-availabilities' in endpoint:
        # Availability/booking classes mock data
        base_response["mock_availability_data"] = {
            "data": [{
                "type": "flight-availability",
                "id": "1",
                "segments": [{
                    "departure": {"iataCode": parameters.get("originLocationCode", "ARN")},
                    "arrival": {"iataCode": parameters.get("destinationLocationCode", "HAN")},
                    "availabilityClasses": [
                        {"class": "Y", "numberOfBookableSeats": 7},
                        {"class": "B", "numberOfBookableSeats": 3},
                        {"class": "M", "numberOfBookableSeats": 2}
                    ]
                }]
            }]
        }
    elif 'pricing' in endpoint:
        # Price confirmation mock data  
        base_response["mock_pricing_data"] = {
            "data": {
                "type": "flight-offers-pricing",
                "flightOffers": [{
                    "price": {"currency": "EUR", "total": "299.00"},
                    "pricingOptions": {"fareType": ["PUBLISHED"]},
                    "validatingAirlineCodes": ["SK"]
                }]
            }
        }
    elif 'flight-orders' in endpoint:
        # Booking mock data
        base_response["mock_booking_data"] = {
            "data": {
                "type": "flight-order",
                "id": "eJzTd9f3NjIwMDe0MjY%3D",
                "queuingOfficeId": "NCE1A0955",
                "flightOffers": [{"id": "1"}],
                "travelers": [{"id": "1"}]
            }
        }
    
    return base_response

In [40]:
# Main Amadeus agent that can use both tools with guardrails
AMADEUS_MAIN_INSTRUCTIONS = """
You are an Amadeus GDS expert that helps users search for flights using natural language.

Your workflow:
1. When user asks about flights, FIRST use parse_flight_query to convert their natural language into structured commands
2. THEN take the result from parse_flight_query, convert it to a JSON string, and pass it to execute_amadeus_query
3. Present the results in a user-friendly format

Crucial Rules:
- Always use parse_flight_query first to structure the user's query
- Convert the parse_flight_query result to a JSON string using json.dumps() before passing to execute_amadeus_query
- The execute_amadeus_query function expects a JSON string parameter
- Explain what defaults were auto-filled (dates, passengers, etc.)
- Present results clearly with key information like prices, times, airlines
- If authentication fails, explain the mock response provided

Example workflow:
1. User: "Find flights from ARN to HAN"
2. Call parse_flight_query("Find flights from ARN to HAN") 
3. Get result_dict = {"query_type": "flight_search", "amadeus_command": {...}}
4. Call execute_amadeus_query(json.dumps(result_dict)) with the JSON string
5. Present the flight results to user (or mock results if auth fails)

IMPORTANT: Convert the parse_flight_query result to JSON string before passing to execute_amadeus_query!
"""

amadeus_main_agent = Agent(
    name="Amadeus Flight Search Agent",
    instructions=AMADEUS_MAIN_INSTRUCTIONS,
    tools=[parse_flight_query, execute_amadeus_query],
    model="gpt-5-mini",
)

message = "What are the booking classes available for SAS flights from Stockholm to London on October 15th, 2025?"

with trace("Amadeus Main"):
    result = await Runner.run(amadeus_main_agent, message)

print(result)


🔍 Analyzing flight query: 'Booking classes for SAS flights from Stockholm to London on October 15th, 2025'
🎯 Intent: booking_classes
🔗 Endpoint: shopping/availability/flight-availabilities
⚡ Method: POST
✅ SDK structure: originDestinations present
✅ SDK structure: travelers present
✅ SDK structure: sources present

🚀 Executing Amadeus API query...
🎯 User intent: booking_classes
📊 Query type: availability_check
🔗 Endpoint: shopping/availability/flight-availabilities
⚡ Method: POST
📋 Parameters: {'originDestinations': [{'id': '1', 'originLocationCode': 'ARN', 'destinationLocationCode': 'LHR', 'departureDateTime': {'date': '2025-10-15'}}], 'travelers': [{'id': '1', 'travelerType': 'ADULT'}], 'sources': ['GDS']}
🔑 Getting new Amadeus access token...
✅ Access token obtained successfully
🔄 Attempt 1/3: Trying v2 API
📡 Making POST request to: https://test.api.amadeus.com/v2/shopping/availability/flight-availabilities
📊 Response status: 404
❌ Attempt 1 failed: 404
🔄 Retrying with different st

In [10]:
# Test the enhanced agentic function tools
print("🧪 Testing Enhanced Agentic Function Tools")
print("=" * 60)

# Test different query types to see endpoint selection
test_queries = [
    "What are the booking classes for Air China between ARN and HAN?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n🔍 Test {i}: {query}")
    try:
        # Test parse function 
        parsed_result = parse_flight_query(query)
        print(f"✅ Parsed successfully")
        print(f"   Intent: {parsed_result.get('user_intent')}")
        print(f"   Endpoint: {parsed_result['amadeus_command']['endpoint']}")
        print(f"   Method: {parsed_result['amadeus_command']['method']}")
        
        # Test execute function with mock data
        execution_result = execute_amadeus_query(json.dumps(parsed_result))
        exec_data = json.loads(execution_result)
        print(f"   Execution: {exec_data.get('status')}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("-" * 40)

print("\n✅ Enhanced agentic function tools are ready!")
print("🎯 They can now handle multiple Amadeus API endpoints intelligently")

🧪 Testing Enhanced Agentic Function Tools

🔍 Test 1: What are the booking classes for Air China between ARN and HAN?
❌ Error: 'FunctionTool' object is not callable
----------------------------------------

✅ Enhanced agentic function tools are ready!
🎯 They can now handle multiple Amadeus API endpoints intelligently


In [ ]:
# Test the optimized parse_flight_query agent with booking classes query
print("🧪 Testing Optimized Parse Agent with Booking Classes Query")
print("=" * 70)

test_query = "What are the booking classes for airline Air China, between ARN-HAN on Oct 10, 2025?"
print(f"🔍 Query: {test_query}")

try:
    # Test the optimized parse function
    parsed_result = parse_flight_query(test_query)
    
    print(f"\n✅ Parse Results:")
    print(f"   Intent: {parsed_result.get('user_intent')}")
    print(f"   Endpoint: {parsed_result['amadeus_command']['endpoint']}")
    print(f"   Method: {parsed_result['amadeus_command']['method']}")
    print(f"   Parameters structure: {type(parsed_result['amadeus_command']['parameters'])}")
    
    # Check if it's the correct structure for flight-availabilities
    params = parsed_result['amadeus_command']['parameters']
    if 'originDestinations' in params:
        print("   ✅ Correct POST body structure with originDestinations")
        print(f"   ✅ Origin: {params['originDestinations'][0]['originLocationCode']}")
        print(f"   ✅ Destination: {params['originDestinations'][0]['destinationLocationCode']}")
        print(f"   ✅ Date: {params['originDestinations'][0]['departureDateTime']['date']}")
        print(f"   ✅ Time: {params['originDestinations'][0]['departureDateTime']['time']}")
    else:
        print("   ❌ Missing originDestinations structure")
    
    if 'travelers' in params:
        print("   ✅ Has travelers array")
    else:
        print("   ❌ Missing travelers array")
        
    if 'sources' in params:
        print("   ✅ Has sources array")
    else:
        print("   ❌ Missing sources array")
    
    # Test execution with the corrected format
    print(f"\n🚀 Testing execution with optimized parameters...")
    execution_result = execute_amadeus_query(json.dumps(parsed_result))
    exec_data = json.loads(execution_result)
    
    print(f"   Execution status: {exec_data.get('status')}")
    if exec_data.get('status') == 'success':
        print("   🎉 SUCCESS! Booking classes query worked!")
        # Check if we got actual availability data
        if 'amadeus_response' in exec_data and 'data' in exec_data['amadeus_response']:
            flight_count = len(exec_data['amadeus_response']['data'])
            print(f"   📊 Found {flight_count} flights with availability data")
        else:
            print("   📄 Got mock/test response")
    else:
        print(f"   ⚠️ Status: {exec_data.get('message', 'Unknown error')}")
    
except Exception as e:
    print(f"❌ Error during test: {e}")

print("\n" + "=" * 70)

In [ ]:
# Test SDK-based parse_flight_query agent
print("🧪 Testing SDK-Based Parse Agent")
print("=" * 50)

test_query = "What are the booking classes available for SAS flights from Stockholm to London on October 15th, 2025?"
print(f"🔍 Query: {test_query}")

try:
    # Test the SDK-based parse function
    parsed_result = parse_flight_query(test_query)
    
    print(f"\n📋 Parse Results:")
    print(f"   Intent: {parsed_result.get('user_intent')}")
    print(f"   Endpoint: {parsed_result['amadeus_command']['endpoint']}")
    print(f"   Method: {parsed_result['amadeus_command']['method']}")
    
    # Verify SDK structure
    params = parsed_result['amadeus_command']['parameters']
    print(f"\n🔍 SDK Structure Validation:")
    
    if 'originDestinations' in params:
        od = params['originDestinations'][0]
        print(f"   ✅ originDestinations: {od['originLocationCode']} → {od['destinationLocationCode']}")
        print(f"   ✅ departureDateTime: {od['departureDateTime']['date']}")
    else:
        print("   ❌ Missing originDestinations")
        
    if 'travelers' in params:
        traveler = params['travelers'][0]
        print(f"   ✅ travelers: {traveler['travelerType']}")
    else:
        print("   ❌ Missing travelers")
        
    if 'sources' in params:
        print(f"   ✅ sources: {params['sources']}")
    else:
        print("   ❌ Missing sources")
    
    # Test execution
    print(f"\n🚀 Testing API execution...")
    execution_result = execute_amadeus_query(json.dumps(parsed_result))
    exec_data = json.loads(execution_result)
    
    print(f"   Status: {exec_data.get('status')}")
    
    if exec_data.get('status') == 'success':
        print("   🎉 SUCCESS! Booking classes API call worked!")
        if 'amadeus_response' in exec_data:
            data = exec_data['amadeus_response'].get('data', [])
            print(f"   📊 Found {len(data)} flight availability results")
            if data:
                # Show first flight's booking classes
                first_flight = data[0]
                if 'segments' in first_flight and first_flight['segments']:
                    segment = first_flight['segments'][0]
                    if 'availabilityClasses' in segment:
                        classes = segment['availabilityClasses']
                        print(f"   🎯 Booking classes: {len(classes)} classes available")
                        for cls in classes[:3]:  # Show first 3
                            print(f"      Class {cls['class']}: {cls['numberOfBookableSeats']} seats")
    else:
        print(f"   ⚠️ Error: {exec_data.get('message', 'Unknown error')}")
        if 'final_status_code' in exec_data:
            print(f"   📊 Final status: {exec_data['final_status_code']}")
    
except Exception as e:
    print(f"❌ Test error: {e}")

print("\n" + "=" * 50)